# HuggingFace Chat Target Testing

This notebook demonstrates the process of testing the HuggingFace Chat Target using various prompts.
The target model will be loaded and interacted with using different prompts to examine its responses.
The goal is to test the model's ability to handle various inputs and generate appropriate and safe outputs.

In [1]:
# Import necessary modules and classes
from pyrit.prompt_target import HuggingFaceChatTarget  
from pyrit.orchestrator import PromptSendingOrchestrator

## Initialize the HuggingFace Chat Target

Here, we initialize the `HuggingFaceChatTarget` with the desired model ID. We will also configure whether to use CUDA for GPU support.

In [ ]:
target = HuggingFaceChatTarget(
    model_id="microsoft/Phi-3-mini-4k-instruct",  # Use the desired model ID
    use_cuda=False,  # Set this to True if you want to use CUDA and have GPU support
    tensor_format="pt",
    verbose=False,
    max_new_tokens=30
)

In [ ]:

# Initialize PromptSendingOrchestrator with HuggingFaceChatTarget
orchestrator = PromptSendingOrchestrator(
    prompt_target=target,  # Use HuggingFaceChatTarget as the prompt target
    verbose=False  # Enable verbose logging for debugging
)

# Use the first few examples (adjust the range as needed)
prompt_list = ["What is 3*3?", "What is 4*4?", "What is 5*5?", "What is 6*6?", "What is 7*7?"]

# Send prompts using the orchestrator
responses = await orchestrator.send_prompts_async(prompt_list=prompt_list)

# Print the conversations and scoring results
orchestrator.print_conversations()  # Use built-in method to display conversations



In [ ]:
another_target = HuggingFaceChatTarget(
    model_id="HuggingFaceTB/SmolLM-135M-Instruct",  # Use the desired model ID
    use_cuda=False,  # Set this to True if you want to use CUDA and have GPU support
    tensor_format="pt",
    verbose=False,
    max_new_tokens=30
)

In [ ]:

# Initialize PromptSendingOrchestrator with HuggingFaceChatTarget
orchestrator = PromptSendingOrchestrator(
    prompt_target=another_target,  # Use HuggingFaceChatTarget as the prompt target
    verbose=False  # Enable verbose logging for debugging
)

# Use the first few examples (adjust the range as needed)
prompt_list = ["What is 3*3?", "What is 4*4?", "What is 5*5?", "What is 6*6?", "What is 7*7?"]

# Send prompts using the orchestrator
responses = await orchestrator.send_prompts_async(prompt_list=prompt_list)

# Print the conversations and scoring results
orchestrator.print_conversations()  # Use built-in method to display conversations